In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import os,sys
sys.path.append('./misc/lib/python3.7/site-packages')

import math
import numpy as np
import requests
import ipywidgets as widgets
%matplotlib notebook
import matplotlib.pyplot as plt
import IPython.display as Display
from IPython.display import display, display_markdown
from ipywidgets import Layout, Label
from pathlib import Path

import parmed as pmd
import re

from scipy.ndimage import gaussian_filter


np.set_printoptions(precision=8)
np.set_printoptions(suppress=True)


HTMLButtonPrompt = '''<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
</head>
<body>
<a href="{link}" target="_blank" >
<button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning" style="width:150px; background-color:#CCCCCC; font-size:10pt; color:black">{text}</button>
</a>
</body>
</html>
'''   

# Width of elements in site parameter grid
grid_width = '2.5cm'
ggap = '0.5cm'
grid_total_width = '12cm'

forbidden_strings = ["..", "/", "\\", " ", "~"]


DATADIR = 'data'


In [16]:


#################################################################
# Spectrum window setup
#################################################################


def format_coord(x, y):
    #return 'x=%.1f, y=%1.2f'%(x, y)
    return ''


def build_axes():
    specax.set_yticks([])
    specax.set_xlabel('$\\bar\\nu$ (cm$^{-1}$)')
    plt.tight_layout()
    plt.show()
        
vaxis = np.arange(-3000, 3000)

specout = widgets.Output(layout=Layout(width='4.25in',
                                      height='4.25in'))



#################################################################
# Spectrum calculation setup
#################################################################


homog_slider = widgets.IntSlider(
    value=10,
    min=5, # min value
    max=50, # max value
    step=1, # exponent step
    description='FWHM (1/cm)',
    disabled=False,
    readout_format='d',
    layout=Layout(width='11cm',
                 margin='5pt')
)
homog_slider.style.description_width='3cm'

homog_fwhm = int(round(homog_slider.value))
def homog_fwhm_on_change(v):
    global homog_fwhm
    homog_fwhm = int(round(v['new']))
homog_slider.observe(homog_fwhm_on_change, names='value')


specfilelbl = widgets.Label(value='Output file prefix: ')

specfile = widgets.Text(
    value='test',
    placeholder='file prefix',
    layout = widgets.Layout(width='5cm'),
    disabled=False
)


exportover_cb = widgets.Checkbox(
    value = False,
    disabled = False,
    description = 'Overwrite Existing?',
    layout = widgets.Layout(width='4cm')
)
exportover_cb.style.description_width='0pt'

export_button = widgets.Button(
    description='Save Now',
    layout=Layout(width='3cm'))

def export_prefix_is_valid():
    prefix = specfile.value
    fbase = DATADIR + "/spec/" + prefix
    
    Suffixes = ['_abs.txt', '_cd.txt']
    for suf in Suffixes:
        fname = fbase + suf
        for chars in forbidden_strings:
            if len(prefix)==0 or prefix.find(chars)!=-1:
                msglbl.value = 'Please enter a valid file prefix for output. Avoid spaces and the special characters "~", "..", "/", and "\\".'
                return False

        if os.path.isfile(fname)==True:
            if exportover_cb.value==False:
                msglbl.value = "Output file exists. Please check 'Overwrite Existing' or pick another export name."
                return False
            else:
                !{"rm " + fname}
            
    return fbase

def export_button_onclick(b):
    fbase = export_prefix_is_valid()
    if fbase!=False:
        np.savetxt(fbase + '_abs.txt', np.vstack([vaxis, abs_line.get_ydata()]).T)
        np.savetxt(fbase + '_cd.txt', np.vstack([vaxis, cd_line.get_ydata()]).T)
        msglbl.value = 'Spectrum exported under prefix ' + fbase
export_button.on_click(export_button_onclick)

homog_lbl = widgets.HTML(
    value = '<p style="font-size:12pt"><b>ZPL Width:</b></p>'
    
)

HomogBox = widgets.Box([homog_lbl, homog_slider], 
                         layout=Layout(flex_flow='column',
                                    width=grid_total_width,
                                    border='solid 0.5pt gray'))


########################################################################
# Log Normal Components
########################################################################


# sdclass is either
# -- 'J' indicating J(w) form
# -- 'C' indicating C(w) form
# where C(w) = w*w*J(w)
#
# func(v, params) is a function that returns the (unnormalized)
# spectral density as a function of v (in cm-1). 
#
# pnames stores the names of each parameter upon which 
# func depends.
#
# pbounds defines the upper and lower bounds for each parameter
class specdens:
    def __init__(self, name, sdclass, func, pnames, pdefs, pbounds):
        self.name = name
        self.sdclass = sdclass
        self.func = func
        self.pnames = pnames
        self.pdefs = pdefs
        self.pbounds = pbounds
        
        
        self.label = widgets.HTML(
            value = '<p style="font-size:12pt"><b>' + name + ':</b></p>'
        )
        
        # This is the widget that displays components
        self.CompsBox = widgets.VBox([])
        
        # This list keeps track of added components
        self.List = []
        
        # This is the add button for this spectral density
        self.add_button = widgets.Button(
            description='Add Component'
        )
        
        def add_component(b):
            newSFloat = widgets.BoundedFloatText(
                value=0.0,
                min=0,
                max=30.0,
                step=0.01,
                description=r'\(S\):',
                disabled=False,
                layout=Layout(width='2.25cm')
            )
            newSFloat.style.description_width='0.5cm'
            
            # This is the function used to delete lognormal components. 
            # It should only be called by the 'X' button defined
            # for each component. 
            def del_on_click(b):
                for item in self.List:
                    if item.children[-1]==b:
                        self.List.remove(item)
                        self.CompsBox.children = tuple(self.List)
            
            newDelBt = widgets.Button(
                description='X',
                layout=Layout(width='2.5em')
            )
            newDelBt.on_click(del_on_click)
            
            wList = [newSFloat]
            for p in range(0, len(self.pnames)):
                newParFloat = widgets.BoundedFloatText(
                    value=self.pdefs[p],
                    min=self.pbounds[p][0],
                    max=self.pbounds[p][1],
                    step=0.01,
                    disabled=False,
                    description=r'\(' + pnames[p] + '\):',
                    layout=Layout(width='4.25cm')
                )
                newParFloat.style.description_width='2.25cm'
                wList.append(newParFloat)
                
            wList.append(newDelBt)
        
            #self.List.append(widgets.HBox([newSFloat, newWcFloat, newSigFloat, newDelBt]))
            self.List.append(widgets.HBox(wList))

            self.CompsBox.children = tuple(self.List)

        
        self.add_button.on_click(add_component)

        # This is the box displayed in the main window. 
        self.Box = widgets.Box([widgets.HBox([self.label, self.add_button]),
              self.CompsBox
             ],
             layout=Layout(flex_flow='column',
                        width=grid_total_width,
                        border='solid 0.5pt gray')
        )


        
# def add_log_normal(b):
    
#     # This is the function used to delete lognormal components. 
#     # It should only be called by the 'X' button defined
#     # for each component. 
#     def del_on_click(b):
#         global LogNormList

#         for item in LogNormList:
#             if item.children[-1]==b:
#                 LogNormList.remove(item)
#                 LogNormCompsBox.children = tuple(LogNormList)


    
#     newWcFloat = widgets.BoundedFloatText(
#         value=0.0,
#         min=0,
#         max=200.0,
#         step=0.01,
#         disabled=False,
#         description=r'\(\bar\nu_{cut}\) (cm\(^{-1}\)):',
#         layout=Layout(width='4.25cm')
#     )
#     newWcFloat.style.description_width='2.25cm'
    
#     newSigFloat = widgets.BoundedFloatText(
#         value=0.0,
#         min=0,
#         max=10.0,
#         step=0.01,
#         description=r'\(\bar\sigma\)  (cm\(^{-1}\)):',
#         disabled=False,
#         layout=Layout(width='4.1cm')
#     )
#     newSigFloat.style.description_width='2.1cm'
    
#     newDelBt = widgets.Button(
#         description='X',
#         layout=Layout(width='2.5em')
#     )
    
#     newDelBt.on_click(del_on_click)
    
    
#     LogNormList.append(widgets.HBox([newSFloat, newWcFloat, newSigFloat, newDelBt]))
    
#     LogNormCompsBox.children = tuple(LogNormList)
    




##############################################################



# LogNormCompsBox = widgets.VBox([])

# LogNormList = []
# def add_log_normal(b):
    
#     # This is the function used to delete lognormal components. 
#     # It should only be called by the 'X' button defined
#     # for each component. 
#     def del_on_click(b):
#         global LogNormList

#         for item in LogNormList:
#             if item.children[-1]==b:
#                 LogNormList.remove(item)
#                 LogNormCompsBox.children = tuple(LogNormList)


#     global LogNormList
#     newSFloat = widgets.BoundedFloatText(
#         value=0.0,
#         min=0,
#         max=30.0,
#         step=0.01,
#         description=r'\(S\):',
#         disabled=False,
#         layout=Layout(width='2.25cm')
#     )
#     newSFloat.style.description_width='0.5cm'
    
#     newWcFloat = widgets.BoundedFloatText(
#         value=0.0,
#         min=0,
#         max=200.0,
#         step=0.01,
#         disabled=False,
#         description=r'\(\bar\nu_{cut}\) (cm\(^{-1}\)):',
#         layout=Layout(width='4.25cm')
#     )
#     newWcFloat.style.description_width='2.25cm'
    
#     newSigFloat = widgets.BoundedFloatText(
#         value=0.0,
#         min=0,
#         max=10.0,
#         step=0.01,
#         description=r'\(\bar\sigma\)  (cm\(^{-1}\)):',
#         disabled=False,
#         layout=Layout(width='4.1cm')
#     )
#     newSigFloat.style.description_width='2.1cm'
    
#     newDelBt = widgets.Button(
#         description='X',
#         layout=Layout(width='2.5em')
#     )
    
#     newDelBt.on_click(del_on_click)
    
    
#     LogNormList.append(widgets.HBox([newSFloat, newWcFloat, newSigFloat, newDelBt]))
    
#     LogNormCompsBox.children = tuple(LogNormList)
    

# addlognorm_bt = widgets.Button(
#     description='Add Component'
# )
# addlognorm_bt.on_click(add_log_normal)

# LogNormBox = widgets.Box([widgets.HBox([lognorm_lbl, addlognorm_bt]),
#               LogNormCompsBox
#              ],
#              layout=Layout(flex_flow='column',
#                         width=grid_total_width,
#                         border='solid 0.5pt gray')
#     )



##############################################################



spec_down = widgets.HTML(HTMLButtonPrompt.format(link=DATADIR + "/spec/", text='Download Spectra'),
        layout=Layout(
            margin='10pt'
        ))

spec_process = widgets.HTML(HTMLButtonPrompt.format(link='Calculator.ipynb', text='Process Spectra'),
        layout=Layout(
            margin='10pt'
        ))



msglbl = widgets.HTML(
    value='',
    layout=Layout(width=grid_total_width)
)

def lndens(v, params):
    vcut = params[0]
    sigma = params[1]
    vals = np.exp(-((np.log(v/vcut))**2)/(2.0*sigma*sigma))*(v>0.0)
    
LogNormal = specdens('Log-Normal', 'J', lndens, 
                     ['\\bar\\nu_{cut}', '\sigma'], 
                     np.array([30.0, 0.6]),
                     np.array([[0.0, 200.0],
                               [0.1, 10.0]]))


display(widgets.HBox([
    widgets.Box([specout,
                widgets.HBox([specfilelbl, specfile, export_button]),
                widgets.HBox([exportover_cb]),
                widgets.HBox([spec_down, spec_process]),
                msglbl
                 ], layout=Layout(
                    flex_flow='column',
                    align_items='center',
                )),
        widgets.Box([
                    widgets.Label(layout=Layout(height='15px')),
                     HomogBox,
#                      LogNormBox,
                     LogNormal.Box
                    ], layout=Layout(flex_flow='column', align_items='flex-start'))
]))

with specout:
    %matplotlib notebook
    specfig = plt.figure(figsize=(4,3))
    specax = plt.gca()
    specax2 = specax.twiny()
    abs_line, = specax.plot(vaxis, 0.0*vaxis)
    cd_line, = specax.plot(vaxis, 0.0*vaxis)
    build_axes()
    
